# Clinical Data Embeddings

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [ ]:
# Loading the dataset
df = pd.read_csv("clinical_Data.csv")

In [ ]:
print("Dataset Overview:\n")
print("Shape of DataFrame:", df.shape)  # Number of rows and columns

In [ ]:
print("\nColumn Names:\n", df.columns.tolist())  # Column names

In [ ]:
print("\nData Types:\n", df.dtypes)  # Data types of each column

In [ ]:
print("\nFirst 5 Rows:\n")  # First 5 rows
df.head()

In [ ]:
# Summary Statistics
print("\nStatistical Summary:\n")  # Summary stats
df.describe(include='all')


In [ ]:
# Missing Values Check
print("\nMissing Values Count:\n")  # Count of missing values per column
df.isna().sum()


In [ ]:
print("\nTotal Missing Values:")  # Total missing values
df.isna().sum().sum()

In [ ]:
# Check for Duplicates
print("\nDuplicate Rows Count:")  # Count of duplicate rows
df.duplicated().sum()

In [ ]:
# Unique Values per Column
print("\nUnique Values Count:\n")  # Unique values per column
df.nunique()

In [ ]:
# Defining the target variable [Censored_0_progressed_1 → 1: Indicates liver cancer and 0: Indicates no progression or absence of liver cancer]
target_column = "Censored_0_progressed_1"
y = df[target_column]
df = df.drop(columns=[target_column])  # Removing target from features

In [ ]:
# Encoding categorical variables
categorical_cols = df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Handling missing values (fill with mean)
df.fillna(df.mean(), inplace=True)

# Normalizing the data
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
#Computing Feature Importance using Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(df_scaled, y)

# feature importances
feature_importance = pd.Series(rf.feature_importances_, index=df_scaled.columns).sort_values(ascending=False)

# Plotting Feature Importance
plt.figure(figsize=(12, 12))
sns.barplot(x=feature_importance.values, y=feature_importance.index)
plt.title("Feature Importance (Random Forest)")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.show()

#  Checking the features having low importance (less than 0.005) in contributing to the target variable
low_importance_threshold = 0.005
low_importance_features = feature_importance[feature_importance < low_importance_threshold].index.tolist()

print("Features to be dropped due to low importance:", low_importance_features)


In [ ]:
# Dropping features that have very low importance
df_selected = df_scaled.drop(columns=low_importance_features)

# Recursive Feature Elimination (RFE)
rfe = RFECV(estimator=rf, step=1, cv=5, scoring='accuracy')
rfe.fit(df_selected, y)


In [ ]:
# Computing correlation matrix
corr_matrix = df_selected.corr()

# Setting threshold for high correlation
threshold = 0.9

# Finding highly correlated features
high_corr_pairs = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            high_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Extracting features to drop (keeping one from each correlated pair)
features_to_drop = set([pair[1] for pair in high_corr_pairs])

# Dropping highly correlated features
df_final_reduced = df_selected.drop(columns=features_to_drop)

#correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False)
plt.title("Feature Correlation Matrix")
plt.show()

#list of dropped features
features_to_drop


In [ ]:
# Normalizing the selected clinical data
scaler = StandardScaler()
df_selected_normalized = scaler.fit_transform(df_final_reduced)  # Shape: (105, num_features)

In [ ]:
# Converting to PyTorch tensor
patient_data = torch.tensor(df_selected_normalized, dtype=torch.float32)  # Shape: (105, num_features)

# Defining PVEM module
class PVEM(nn.Module):
    def __init__(self, input_dim, embedding_dim=128):
        super(PVEM, self).__init__()
        self.weight = nn.Parameter(torch.randn(input_dim, embedding_dim))  # Learnable transformation
        self.bias = nn.Parameter(torch.randn(input_dim, embedding_dim))  # Feature-wise bias
    
    def forward(self, x):
        return x.unsqueeze(-1) * self.weight + self.bias  # Output shape: (105, num_features, 128)


In [ ]:
# Initializing PVEM model
num_features = patient_data.shape[1]  # Number of selected features
embedding_dim = 128
model = PVEM(num_features, embedding_dim)

# Generating embeddings using PVEM
embeddings = model(patient_data).detach().numpy()  # Shape: (105, num_features, 128)

In [ ]:
print("Embeddings Generated Successfully!")
print("Final Shape:", embeddings.shape)  # Expected: (105, num_features, 128)

In [ ]:
embeddings.mean()

In [ ]:
embeddings.std()

In [ ]:
embeddings.shape

In [ ]:
print("Embedding of first patient (first feature):\n", embeddings[0, 0])  # Shape: (128,)
print("Embedding of first patient (second feature):\n", embeddings[0, 1])  # Shape: (128,)


In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(embeddings[0], cmap="coolwarm", annot=False)
plt.title("Heatmap of First Patient's Feature Embeddings")
plt.xlabel("Embedding Dimension (128)")
plt.ylabel("Feature Index")
plt.show()


In [ ]:
# Flatten embeddings to (105, num_features * 128)
flat_embeddings = embeddings.reshape(105, -1)

# Compute similarity matrix
similarity_matrix = cosine_similarity(flat_embeddings)

# Extract only the first 5 patients' similarities
similarity_subset = similarity_matrix[:5, :5]

# Plot heatmap
plt.figure(figsize=(6, 4))  # Adjust size
ax = sns.heatmap(similarity_subset, annot=True, cmap="coolwarm", linewidths=0.5, fmt=".2f")

# Labels
plt.title("Cosine Similarity Heatmap (First 5 Patients)", fontsize=12)
plt.xticks(ticks=range(5), labels=[f"P{i+1}" for i in range(5)], fontsize=10)
plt.yticks(ticks=range(5), labels=[f"P{i+1}" for i in range(5)], fontsize=10, rotation=0)

plt.show()


In [ ]:
# Saving embeddings as NumPy file
np.save("patient_feature_embeddings.npy", embeddings)

In [ ]:
reshaped_data = embeddings.reshape(-1, 128)

# Creating a multi-index for (105, 38) patients & features
index = pd.MultiIndex.from_product([range(105), range(38)], names=["Patient", "Feature"])
df = pd.DataFrame(reshaped_data, index=index)

# Saving as CSV
df.to_csv("output_multiindex.csv")

print("CSV saved with shape:", df.shape)